In [10]:
#首先将用到的包进行导入
import pandas as pd
import numpy as np
import jieba
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection 
from sklearn import preprocessing

In [11]:
#将数据进行读取
data=pd.read_csv('./reviews_score_update.csv',index_col=0)
data.head()

,评论,评分
0,问责 领导 上 黄镇 党委书记 张涛 宣国 才 真能 一手遮天 几天 看 有人 举报 施 某...,2
1,江歌 事件 教会 孩子 善良 更要 懂得 保护 ! 过去 一年 江歌 悲剧 几日 再次 刷屏...,1
2,绝味 鸭 脖 广告 开 黄腔 引 众怒 双 11 拼值 双 11 1600 亿 销售额 ...,2
3,央视 曝光 ! 如东 医药企业 槽罐车 改成 垃圾车 夜间 偷排 高浓度 废水 2016 年...,2
4,恶劣 至极 央视 都 曝光 ! 南通 如东 医药企业 槽罐车 改成 洒水车 夜间 偷排 高浓...,2


In [12]:
#现在是划分数据集
#random_state 取值，这是为了在不同环境中，保证随机数取值一致，以便验证模型的实际效果。
train_x,test_x,train_y,test_y=model_selection.train_test_split(data.评论.values.astype('U'),data.评分.values,test_size=0.2,random_state=1)
 
#划分完毕，查看数据形状
print(train_x.shape,test_x.shape)
#train_x 训练集数据 test_x 测试集数据  train_y训练集的标签 test_y 测试集的标签

(5872,) (1468,)


In [13]:
#定义函数，从哈工大中文停用词表里面，把停用词作为列表格式保存并返回 在这里加上停用词表是因为TfidfVectorizer和CountVectorizer的函数中
#可以根据提供用词里列表进行去停用词
def get_stopwords(stop_word_file):
    with open(stop_word_file,encoding='UTF-8') as f:
        stopwords=f.read()
    stopwords_list=stopwords.split('\n')
    custom_stopwords_list=[i for i in stopwords_list]
    return custom_stopwords_list

In [14]:
#获得由停用词组成的列表
stop_words_file = './hit_stopwords.txt'
stopwords = get_stopwords(stop_words_file)


In [15]:
stopwords

['———',
 '》），',
 '）÷（１－',
 '”，',
 '）、',
 '＝（',
 ':',
 '→',
 '℃ ',
 '&',
 '*',
 '一一',
 '~~~~',
 '’',
 '. ',
 '『',
 '.一',
 './',
 '-- ',
 '』',
 '＝″',
 '【',
 '［＊］',
 '｝＞',
 '［⑤］］',
 '［①Ｄ］',
 'ｃ］',
 'ｎｇ昉',
 '＊',
 '//',
 '［',
 '］',
 '［②ｅ］',
 '［②ｇ］',
 '＝｛',
 '}',
 '，也 ',
 '‘',
 'Ａ',
 '［①⑥］',
 '［②Ｂ］ ',
 '［①ａ］',
 '［④ａ］',
 '［①③］',
 '［③ｈ］',
 '③］',
 '１． ',
 '－－ ',
 '［②ｂ］',
 '’‘ ',
 '××× ',
 '［①⑧］',
 '０：２ ',
 '＝［',
 '［⑤ｂ］',
 '［②ｃ］ ',
 '［④ｂ］',
 '［②③］',
 '［③ａ］',
 '［④ｃ］',
 '［①⑤］',
 '［①⑦］',
 '［①ｇ］',
 '∈［ ',
 '［①⑨］',
 '［①④］',
 '［①ｃ］',
 '［②ｆ］',
 '［②⑧］',
 '［②①］',
 '［①Ｃ］',
 '［③ｃ］',
 '［③ｇ］',
 '［②⑤］',
 '［②②］',
 '一.',
 '［①ｈ］',
 '.数',
 '［］',
 '［①Ｂ］',
 '数/',
 '［①ｉ］',
 '［③ｅ］',
 '［①①］',
 '［④ｄ］',
 '［④ｅ］',
 '［③ｂ］',
 '［⑤ａ］',
 '［①Ａ］',
 '［②⑧］',
 '［②⑦］',
 '［①ｄ］',
 '［②ｊ］',
 '〕〔',
 '］［',
 '://',
 '′∈',
 '［②④',
 '［⑤ｅ］',
 '１２％',
 'ｂ］',
 '...',
 '...................',
 '…………………………………………………③',
 'ＺＸＦＩＴＬ',
 '［③Ｆ］',
 '」',
 '［①ｏ］',
 '］∧′＝［ ',
 '∪φ∈',
 '′｜',
 '｛－',
 '②ｃ',
 '｝',
 '［③①］',
 'Ｒ．Ｌ．',
 '［①Ｅ］',
 'Ψ',
 '－［＊］－',
 '↑',
 '

In [16]:
'''
使用TfidfVectorizer()和 CountVectorizer()分别对数据进行特征的提取，投放到不同的模型中进行实验
'''
#开始使用TF-IDF进行特征的提取，对分词后的中文语句做向量化。
#引进TF-IDF的包
TF_Vec=TfidfVectorizer(max_df=0.8,
                       min_df = 3,
                       stop_words=frozenset(stopwords)
                      )
#拟合数据，将数据准转为标准形式，一般使用在训练集中
train_x_tfvec=TF_Vec.fit_transform(train_x)
#通过中心化和缩放实现标准化，一般使用在测试集中
test_x_tfvec=TF_Vec.transform(test_x)
 
#开始使用CountVectorizer()进行特征的提取。它依据词语出现频率转化向量。并且加入了去除停用词
CT_Vec=CountVectorizer(max_df=0.8,#在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
                       min_df = 3,#在低于这一数量的文档中出现的关键词（过于独特），去除掉。
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',#使用正则表达式，去除想去除的内容
                       stop_words=frozenset(stopwords))#加入停用词)
#拟合数据，将数据转化为标准形式，一般使用在训练集中
train_x_ctvec=CT_Vec.fit_transform(train_x)
#通过中心化和缩放实现标准化，一般使用在测试集中
test_x_ctvec=CT_Vec.transform(test_x)

c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '

In [44]:
'''
使用TF_IDF提取的向量当作数据特征传入模型
'''
#构建模型之前首先将包进行导入
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split, GridSearchCV
import time
start_time=time.time()
#创建模型
lr = linear_model.LogisticRegression(penalty='l2', C=1, solver='liblinear', max_iter=1000, multi_class='ovr')
#进行模型的优化，因为一些参数是不确定的，所以就让模型自己在训练中去确定自己的参数 模型的名字也由LR转变为model
model = GridSearchCV(lr, cv=3, param_grid={
        'C': np.logspace(0, 4, 30),
        'penalty': ['l1', 'l2']
    })
#模型拟合tf-idf拿到的数据
model.fit(train_x_tfvec,train_y)
#查看模型自己拟合的最优参数
print('最优参数：', model.best_params_)
#在训练集上的正确率
train_accracy=accuracy_score(train_y,pre_train_y)
#训练结束查看预测 输入测试集查看预测
pre_test_y=model.predict(test_x_tfvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(test_y,pre_test_y)
F1_score=f1_score(test_y,pre_test_y,average='macro')
Recall_score=recall_score(test_y,pre_test_y,average='macro')
Precision_score=precision_score(test_y,pre_test_y,average='macro')
print('使用TF-IDF提取特征使用逻辑回归,让模型自适应参数，进行模型优化\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
print("Precision_score",Precision_score)
print("Recall_score",Recall_score)
print("F1_score",F1_score)
end_time=time.time()
print("")
print("使用模型优化的程序运行时间为",end_time-start_time)

最优参数： {'C': 1.8873918221350972, 'penalty': 'l2'}
使用TF-IDF提取特征使用逻辑回归,让模型自适应参数，进行模型优化
训练集:0.9967643051771117
测试集:0.6995912806539509
Precision_score 0.6410988473480499
Recall_score 0.5153710968208768
F1_score 0.4940038081977776

使用模型优化的程序运行时间为 162.13877081871033


In [50]:
#使用KNN模型
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
start_time=time.time()
#创建模型
Kn = KNeighborsClassifier()
#拟合从tf-idf拿到的数据
Kn.fit(train_x_tfvec,train_y)
#在训练时查看训练集的准确率
pre_train_y=Kn.predict(train_x_tfvec)
#在训练集上的正确率
train_accracy=accuracy_score(train_y,pre_train_y)
#训练结束查看预测 输入测试集查看预测
pre_test_y=Kn.predict(test_x_tfvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(test_y,pre_test_y)
F1_score=f1_score(test_y,pre_test_y,average='macro')
Recall_score=recall_score(test_y,pre_test_y,average='macro')
Precision_score=precision_score(test_y,pre_test_y,average='macro')
print('使用TfidfVectorizer提取特征使用KNN分类器的准确率\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
print("Precision_score",Precision_score)
print("Recall_score",Recall_score)
print("F1_score",F1_score)
end_time=time.time()
print("使用KNN分类器的程序运行时间为",end_time-start_time)

使用TfidfVectorizer提取特征使用KNN分类器的准确率
训练集:0.5279291553133515
测试集:0.4993188010899183
Precision_score 0.43800697684192835
Recall_score 0.3396952489510716
F1_score 0.2395554676705456
使用KNN分类器的程序运行时间为 2.509584903717041


In [49]:
### Random Forest Classifier 随机森林分类器 
from sklearn.ensemble import RandomForestClassifier 
import time
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import roc_curve, auc  ###计算roc和auc
start_time=time.time()
#创建模型
Rfc = RandomForestClassifier(n_estimators=8)
#拟合从CounterfVectorizer拿到的数据
Rfc.fit(train_x_ctvec,train_y)
#在训练时查看训练集的准确率
pre_train_y=Rfc.predict(train_x_ctvec)
#在训练集上的正确率
train_accracy=accuracy_score(train_y,pre_train_y)
#训练结束查看预测 输入测试集查看预测
pre_test_y=Rfc.predict(test_x_ctvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(test_y,pre_test_y)
F1_score=f1_score(test_y,pre_test_y,average='macro')
Recall_score=recall_score(test_y,pre_test_y,average='macro')
Precision_score=precision_score(test_y,pre_test_y,average='macro')
print('使用CounterfVectorizer提取特征使用随机森林分类器的准确率\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
print("Precision_score",Precision_score)
print("Recall_score",Recall_score)
print("F1_score",F1_score)
end_time=time.time()
print("使用随机森林分类器的程序运行时间为",end_time-start_time)

使用CounterfVectorizer提取特征使用随机森林分类器的准确率
训练集:0.9737738419618529
测试集:0.646457765667575
Precision_score 0.5235176381727839
Recall_score 0.48359791430956384
F1_score 0.47500289504660415
使用随机森林分类器的程序运行时间为 1.387753963470459


In [35]:
### Decision Tree Classifier  决策树
from sklearn import tree
import time
start_time=time.time()
#创建模型
Rf = tree.DecisionTreeClassifier()
#拟合从tf-idf拿到的数据
Rf.fit(train_x_tfvec,train_y)
#在训练时查看训练集的准确率
pre_train_y=Rf.predict(train_x_tfvec)
#在训练集上的正确率
train_accracy=accuracy_score(train_y,pre_train_y)
#训练结束查看预测 输入测试集查看预测
pre_test_y=Rfc.predict(test_x_ctvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(test_y,pre_test_y)
F1_score=f1_score(test_y,pre_test_y,average='macro')
Recall_score=recall_score(test_y,pre_test_y,average='macro')
Precision_score=precision_score(test_y,pre_test_y,average='macro')
print('使用tf提取特征使用决策树分类器的准确率\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
print("Precision_score",Precision_score)
print("Recall_score",Recall_score)
print("F1_score",F1_score)
end_time=time.time()
print("使用决策树分类器的程序运行时间为",end_time-start_time)

使用tf提取特征使用决策树分类器的准确率
训练集:0.9967643051771117
测试集:0.6362397820163488
Precision_score 0.5232498515679819
Recall_score 0.47662469387797274
F1_score 0.47009327498902276
使用决策树分类器的程序运行时间为 8.262097358703613


In [42]:
### 贝叶斯
from sklearn.naive_bayes import MultinomialNB
import time
start_time=time.time()
#创建模型
Bys = MultinomialNB()
#拟合数据
Bys.fit(train_x_ctvec, train_y)# 学习,拟合模型
#在训练集上的正确率
train_accracy=accuracy_score(train_y,pre_train_y)
#训练结束查看预测 输入测试集查看预测
pre_test_y=Bys.predict(test_x_ctvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(test_y,pre_test_y)
F1_score=f1_score(test_y,pre_test_y,average='macro')
Recall_score=recall_score(test_y,pre_test_y,average='macro')
Precision_score=precision_score(test_y,pre_test_y,average='macro')
print('使用CounterVectorizer提取特征使用贝叶斯分类器的准确率\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
print("Precision_score",Precision_score)
print("Recall_score",Recall_score)
print("F1_score",F1_score)
end_time=time.time()
print("使用贝叶斯分类器的程序运行时间为",end_time-start_time)

使用CounterVectorizer提取特征使用贝叶斯分类器的准确率
训练集:0.9967643051771117
测试集:0.6185286103542235
Precision_score 0.5399800928306252
Recall_score 0.5584755922961474
F1_score 0.5401647116167149
使用贝叶斯分类器的程序运行时间为 0.01701211929321289


In [52]:
#使用SVM分类器
from sklearn.svm import SVC
import time
start_time=time.time()
#创建模型
SVM = SVC(C=1.0, kernel='rbf', gamma='auto')
#拟合数据
SVM.fit(train_x_ctvec, train_y)# 学习,拟合模型
#在训练集上的正确率
train_accracy=accuracy_score(train_y,pre_train_y)
#训练结束查看预测 输入测试集查看预测
pre_test_y=SVM.predict(test_x_ctvec)
#查看在测试集上的准确率
test_accracy = accuracy_score(test_y,pre_test_y)
F1_score=f1_score(test_y,pre_test_y,average='macro')
Recall_score=recall_score(test_y,pre_test_y,average='macro')
Precision_score=precision_score(test_y,pre_test_y,average='macro')
print('使用CounterfVectorizer提取特征使用SVM分类器的准确率\n训练集:{0}\n测试集:{1}'.format(train_accracy,test_accracy))
print("Precision_score",Precision_score)
print("Recall_score",Recall_score)
print("F1_score",F1_score)
end_time=time.time()
print("使用SVM分类器的程序运行时间为",end_time-start_time)

使用CounterfVectorizer提取特征使用SVM分类器的准确率
训练集:0.5279291553133515
测试集:0.5088555858310627
Precision_score 0.3929472766484651
Recall_score 0.3484851644112679
F1_score 0.26324953973988796
使用SVM分类器的程序运行时间为 50.71697735786438


c:\Users\a\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
